In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import sqlite3

In [4]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()

In [5]:
df_store = pd.read_sql_query("SELECT * FROM api_store", con)
df_review = pd.read_sql_query("SELECT * FROM api_review", con)

In [6]:
df_store.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0


In [7]:
df_review.head()

,id,total_score,content,reg_time,store_id,user_id
0,1,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,2020-04-07 07:13:49.444324,15,68632
1,2,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,2020-04-07 07:13:49.444324,18,389728
2,3,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,2020-04-07 07:13:49.444324,19,68716
3,4,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",2020-04-07 07:13:49.444324,37,774353
4,5,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2020-04-07 07:13:49.444324,38,115682


In [8]:
con.close()

In [11]:
stores = df_store[['id', 'store_name', 'category']]
reviews = df_review[['user_id', 'store_id', 'total_score']]
stores.rename(columns={'id': 'store_id'}, inplace=True)
reviews.rename(columns={'total_score': 'rating'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
df = pd.merge(reviews, stores, on="store_id")
df.head()

,user_id,store_id,rating,store_name,category
0,68632,15,5.0,써리힐,호주레스토랑
1,389728,18,5.0,진삼미 샌드위치,샌드위치|쥬스
2,68716,19,4.0,한옥마을 전주비빔밥,전주비빔밥
3,774353,37,2.0,007식당,굴비정식|굴비
4,115682,38,3.0,010수산,횟집


In [14]:
df.shape

(91398, 5)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91398 entries, 0 to 91397
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     91398 non-null  int64  
 1   store_id    91398 non-null  int64  
 2   rating      91398 non-null  float64
 3   store_name  91398 non-null  object 
 4   category    91398 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 4.2+ MB


## Ratings Distribution

In [16]:
!pip install plotly

In [31]:
reader = surprise.Reader(rating_scale=(0, 5))
dataframe = surprise.Dataset.load_from_df(df[['user_id', 'store_id', 'rating']], reader)
trainset = dataframe.build_full_trainset()
global_mean = trainset.global_mean
bu = np.zeros(trainset.n_users)
bi = np.zeros(trainset.n_items)

In [32]:
reg = 0.02 # reg: 정규화 가중치
learning_rate = 0.005 # learning_rate: 최적화를 위한 스탭 사이즈
n_epochs = 20 # n_epochs: 최적화 반복 횟수

def bsl_option_is_sgd():
    for dummy in range(n_epochs):
        for u, i, r in self.trainset.all_ratings():
            err = (r - (global_mean + bu[u] + bi[i]))
            bu[u] += lr * (err - reg * bu[u])
            bi[i] += lr * (err - reg * bi[i])
    return bu, bi

In [33]:
reg_i = 10 # reg_i: 상품에 대한 가중치
reg_u = 15 # reg_u: 유저에 대한 가중치
n_epochs = 10 # n_epochs: 최적화 반복 횟수

def bsl_option_is_als():
    for dummy in range(n_epoch):
        for i in trainset.all_items():
            dev_i = 0
            for (u, r) in trainset.ir[i]:
                dev_i += r - global_mean - bu[u]

            bi[i] = dev_i / (reg_i + len(trainset.ir[i]))

        for u in trainset.all_users():
            dev_u = 0
            for (i, r) in trainset.ur[u]:
                dev_u += r - global_mean - bi[i]

            bu[u] = dev_u / (reg_u + len(trainset.ur[u]))
    return bu, bi

In [43]:
sim_options = {
    'name': 'pearson', # 사용할 유사도 이름. ['cosine', 'pearson_baseline', 'pearson']
    'user_based': True, # user를 base로 할것인지 item을 base로 할 것인지 결정.
    # u, v간 교집합 원소 최소 갯수를 설정한다. 이 경우는 5개 이상의 공통된 아이템(or 사용자)가
    # 존재하면 유사도를 계산하지만, 그 미만의 경우에는 sim(u, v) == 0로 설정한다.
    'min_support': 5,
    'shrinkage': 0, # 이 패러미터는 pearson_baseline을 사용할 경우에만 해당된다.
}
algo = surprise.KNNBasic(sim_options=sim_options)

In [47]:
from surprise.model_selection import cross_validate

In [48]:
cross_validate(algo, data)["test_mae"].mean()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


0.8527115999606029

In [42]:
data = df.pivot_table('rating', index='user_id', columns='store_id')

In [43]:
data = data.fillna(0)

In [44]:
data.head()

store_id,15,18,19,37,38,49,50,53,56,58,...,360481,360482,360486,360489,360492,360493,360499,360505,360514,360515
user_id,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
data.shape

(18992, 45700)

In [46]:
data = data.transpose()
data.head(2)

user_id,7,15,23,62,64,74,105,137,147,156,...,949436,949554,949596,949670,949944,949946,949951,950224,950225,950331
store_id,,,,,,,,,,,,,,,,,,,,,
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
data_sim = cosine_similarity(data, data)
print(data_sim.shape)

MemoryError: Unable to allocate 6.47 GiB for an array with shape (45700, 18992) and data type float64

In [50]:
from math import sqrt

In [51]:
# 피어슨 상관계수 구하기
def sim_pearson(data, name1, name2):
    sumX=0 # X의 합
    sumY=0 # Y의 합
    sumPowX=0 # X 제곱의 합
    sumPowY=0 # Y 제곱의 합
    sumXY=0 # X*Y의 합
    count=0 #영화 개수
    
    for i in data[name1]: # i = key
        if i in data[name2]: # 같은 영화를 평가했을때만
            sumX+=data[name1][i]
            sumY+=data[name2][i]
            sumPowX+=pow(data[name1][i],2)
            sumPowY+=pow(data[name2][i],2)
            sumXY+=data[name1][i]*data[name2][i]
            count+=1
    
    return ( sumXY- ((sumX*sumY)/count) )/ sqrt( (sumPowX - (pow(sumX,2) / count)) * (sumPowY - (pow(sumY,2)/count)))

In [52]:
# 딕셔너리 돌면서 상관계수순으로 정렬
def top_match(data, name, index=3, sim_function=sim_pearson):
    li=[]
    for i in data: #딕셔너리를 돌고
        if name!=i: #자기 자신이 아닐때만
            li.append((sim_function(data,name,i),i)) #sim_function()을 통해 상관계수를 구하고 li[]에 추가
    li.sort() #오름차순
    li.reverse() #내림차순
    return li[:index]

In [ ]:
def getRecommendation (data,person,sim_function=sim_pearson):
    result = top_match(critics, person ,len(data))
    
    simSum=0 # 유사도 합을 위한 변수
    score=0 # 평점 합을 위한 변수
    li=[] # 리턴을 위한 리스트
    score_dic={} # 유사도 총합을 위한 dic
    sim_dic={} # 평점 총합을 위한 dic
 
    for sim,name in result: # 튜플이므로 한번에 
        if sim<0 : continue #유사도가 양수인 사람만
        for movie in data[name]: 
            if movie not in data[person]: #name이 평가를 내리지 않은 영화
                score+=sim*data[name][movie] # 그사람의 영화평점 * 유사도
                score_dic.setdefault(movie,0) # 기본값 설정
                score_dic[movie]+=score # 합계 구함
 
                # 조건에 맞는 사람의 유사도의 누적합을 구한다
                sim_dic.setdefault(movie,0) 
                sim_dic[movie]+=sim
 
            score=0  #영화가 바뀌었으니 초기화한다
    
    for key in score_dic: 
        score_dic[key]=score_dic[key]/sim_dic[key] # 평점 총합/ 유사도 총합
        li.append((score_dic[key],key)) # list((tuple))의 리턴을 위해서.
    li.sort() #오름차순
    li.reverse() #내림차순
    return li